In [4]:
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

# Number of samples
num_samples = 1000

# Simulate Payload (Ton)
payload = np.random.normal(loc=145.205, scale=17.082, size=num_samples)

# Simulate Round-trip Distance (km)
round_trip_distance = np.random.normal(loc=5, scale=1, size=num_samples)  # Example values

# Simulate Number of Cycles/Shift
cycles_per_shift = np.random.normal(loc=1, scale=1, size=num_samples)  # Example values

# Simulate Total Hours of Operation/Shift
total_hours_operation = np.random.normal(loc=8, scale=1, size=num_samples)  # Example values

# Calculate AWSS using the provided formula
awss = (round_trip_distance * cycles_per_shift) / total_hours_operation

# Simulate Temperature (°C)
temperature = np.random.normal(loc=30, scale=5, size=num_samples)  # Assuming a range around 25-35°C

# Simulate Tire Pressure (psi) with some anomalies
tire_pressure = np.random.normal(loc=100, scale=10, size=num_samples)

# Introduce anomalies
anomalies = np.random.choice([0, 1], size=num_samples, p=[0.95, 0.05])  # 5% chance of being an anomaly
tire_pressure = np.where(anomalies == 1, 
                         np.random.choice([np.random.uniform(60, 85), np.random.uniform(110, 140)], size=num_samples), 
                         tire_pressure)

# Ensure tire pressure is within a reasonable range
tire_pressure = np.clip(tire_pressure, 60, 140)

# Create the anomaly column based on tire pressure thresholds
anomaly = np.where((tire_pressure < 85) | (tire_pressure > 110), 1, 0)

# Create the DataFrame
df = pd.DataFrame({
    'Payload (Ton)': payload,
    'Round-trip Distance (km)': round_trip_distance,
    'Cycles/Shift': cycles_per_shift,
    'Total Hours Operation/Shift': total_hours_operation,
    'AWSS': awss,
    'Tire Pressure (psi)': tire_pressure,
    'Temperature (°C)': temperature,
    'Anomaly': anomaly
})

# Save the dataset to a CSV file
df.to_csv('dumper_tire_anomaly_detection.csv', index=False)

# Display the first few rows
df.head()


,Payload (Ton),Round-trip Distance (km),Cycles/Shift,Total Hours Operation/Shift,AWSS,Tire Pressure (psi),Temperature (°C),Anomaly
0,153.689871,6.399355,0.324822,6.092192,0.341199,95.762403,25.682532,0
1,142.843169,5.924634,0.855481,7.139615,0.709900,95.465859,29.843983,0
2,156.268816,5.059630,0.207580,7.586394,0.138442,82.043568,30.090084,1
3,171.221396,4.353063,0.692038,9.887688,0.304671,96.699098,32.363152,0
4,141.205192,5.698223,-0.893615,8.556553,-0.595101,107.328291,23.165708,0
